In [144]:
import numpy as np
import os

from  PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image
import splitfolders

from tensorflow.python.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Dropout, Flatten, Dense

from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
import keras

from tensorflow.python.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.callbacks import TensorBoard
import time



input_folder='.\\data_concreate\\D\\'
output_folder='\\data_concreate\\images\\'
InpShape=32

In [22]:


# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio(input_folder, output=output_folder,
#       seed=42, ratio=(.7, .2, .1), group_prefix=None, move=False) # default values



Copying files: 17776 files [00:20, 850.37 files/s] 


In [6]:
base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(InpShape, InpShape, 3), # VGG16 expects min 32 x 32
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False


In [192]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      #keras.metrics.CategoricalCrossentropy(name='ccent'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
   
      
      #keras.metrics.AUC(name='auc'),
     # keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]



In [279]:
inputs = tf.keras.Input(shape=(InpShape,InpShape,3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2,2))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x=tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)







In [280]:
model.summary()

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_106 (Conv2D)         (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_94 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_107 (Conv2D)         (None, 13, 13, 32)        9248      
                                                                 
 max_pooling2d_95 (MaxPoolin  (None, 6, 6, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_108 (Conv2D)         (None, 4, 4, 64)          184

In [281]:

NAME="cement_binary-{}".format(int(time.time())) # log file name for tensorboard


model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(), # default from_logits=False
              metrics=METRICS)



early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1,mode='auto')
learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto')
tensorboard=TensorBoard(log_dir='./logs/{}'.format(NAME)) ## logs are under logs folder for tensorboard


batch_size = 32 
epochs = 75







In [270]:



train_datagen = ImageDataGenerator(rescale = 1./255
                                  )
training_set = train_datagen.flow_from_directory('./'+output_folder + 'train',
                                                 target_size = (InpShape,InpShape),
                                                 batch_size = batch_size,
                                                 
                                                 class_mode = 'binary')


Found 12442 images belonging to 2 classes.


In [181]:
training_set.class_indices

{'NO': 0, 'OK': 1}

In [271]:
val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory('./'+output_folder +'val',
                                    
                                            target_size = (InpShape, InpShape),
                                            batch_size = batch_size,
                                            class_mode = 'binary')


Found 3554 images belonging to 2 classes.


In [182]:
val_set.class_indices

{'NO': 0, 'OK': 1}

In [282]:
history = model.fit(training_set, validation_data=val_set, 
                     epochs = epochs,callbacks = [learning_rate, early_stop, tensorboard])

Epoch 1/75
389/389 [==============================] - 17s 39ms/step - loss: 0.5881 - accuracy: 0.7276 - precision: 0.7259 - recall: 0.7693 - tp: 5719.0000 - fp: 2159.0000 - tn: 4629.0000 - fn: 1715.0000 - val_loss: 0.5645 - val_accuracy: 0.7316 - val_precision: 0.8188 - val_recall: 0.6249 - val_tp: 1161.0000 - val_fp: 257.0000 - val_tn: 1439.0000 - val_fn: 697.0000 - lr: 0.0010
Epoch 2/75
389/389 [==============================] - 14s 36ms/step - loss: 0.5489 - accuracy: 0.7495 - precision: 0.7652 - recall: 0.7512 - tp: 4886.0000 - fp: 1499.0000 - tn: 4439.0000 - fn: 1618.0000 - val_loss: 0.5124 - val_accuracy: 0.7794 - val_precision: 0.7609 - val_recall: 0.8428 - val_tp: 1566.0000 - val_fp: 492.0000 - val_tn: 1204.0000 - val_fn: 292.0000 - lr: 0.0010
Epoch 3/75
389/389 [==============================] - 14s 36ms/step - loss: 0.5143 - accuracy: 0.7745 - precision: 0.7849 - recall: 0.7832 - tp: 5094.0000 - fp: 1396.0000 - tn: 4542.0000 - fn: 1410.0000 - val_loss: 0.4668 - val_accuracy: 

In [283]:
test_datagen = ImageDataGenerator(rescale = 1./255)


test_set = test_datagen.flow_from_directory('./'+output_folder +'test',
                                            target_size = (InpShape, InpShape),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 1780 images belonging to 2 classes.


In [284]:
#history.model.evaluate(test_datagen)
score = model.evaluate(test_set)

#score = model.evaluate(generator=test_set)
#print('Accuracy:', score[1])

score


56/56 [==============================] - 2s 29ms/step - loss: 0.3573 - accuracy: 0.8478 - precision: 0.8359 - recall: 0.8817 - tp: 820.0000 - fp: 161.0000 - tn: 689.0000 - fn: 110.0000


[0.3572673797607422,
 0.8477528095245361,
 0.835881769657135,
 0.8817204236984253,
 820.0,
 161.0,
 689.0,
 110.0]

In [285]:
model_json = model.to_json()
with open("modelbm.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("modelbm.h5")